In [34]:
import numpy as np
import pandas as pd
from pandas import json_normalize 
import requests
import json
import time as time
from datetime import date
from datetime import timedelta

#SO list
file_name_SO = 'orders-15-10-2022.csv'
col = ['Cin7 Status','Created','Stage','Ref','Total Incl','Inv No',	'Invoice',	'Paid',	'Company',	'Customer',	'Dispatched',	'Items',	'Sales Creator',	'Internal Comments']

df_SO_list = pd.read_csv(file_name_SO, usecols=col)

#SO detail
file_SO_detail = 'SO-'+date.today().strftime("%d-%m-%Y")+'.csv'

col1 = ['Order Ref','Item Qty Moved']

df_SO_detail = pd.read_csv(file_SO_detail, usecols=col1)
table = pd.pivot_table(df_SO_detail, values='Item Qty Moved', index=['Order Ref'], aggfunc=np.sum, fill_value=0)


#remove imported invoice

df_SO_not_imported = df_SO_list.loc[df_SO_list['Invoice']!='Imported'] 

#convert the paid column

df_SO_not_imported.loc[:,'Paid'] = df_SO_not_imported.loc[:,'Paid'].astype(str).apply(lambda x:x.replace('<font color=#009788><b>PAID: ','').replace('%</b></font>','').replace(' ','').replace('-','').replace('(','').replace(')','').replace('/','').replace('\t','').lower())

df_SO_not_imported['Paid'] = df_SO_not_imported['Paid'].astype(float)
df_SO_not_imported['Paid'] = df_SO_not_imported['Paid'].fillna(0)

#convert the status
df_SO_not_imported.loc[:,'Cin7 Status'] = df_SO_not_imported.loc[:,'Cin7 Status'].astype(str).apply(lambda x:x.replace('<font color=#009788>','').replace('</font>','').replace('<fontcolor=red>','').replace('<fontcolor=#2196f3>','').replace('<font color=#2196f3>','').replace('<font color=red>','').replace('/','').replace('\t','').lower())

#df_SO_not_imported.rename(columns = {'Paid':'Paid (%)'}, inplace = True)

#Remove fully paid

df_SO_unpaid = df_SO_not_imported.loc[df_SO_not_imported.Paid <100, : ]
df_SO_unpaid.set_index('Ref', inplace=True)
df_SO_unpaid['Qty Moved'] = df_SO_unpaid.index.map(table['Item Qty Moved'])
df_SO_unpaid = df_SO_unpaid.loc[df_SO_unpaid["Total Incl"]>0]

wholesaler = ['Auckland Camera Centre','Carters Photographics','Dove Electronics Limited','Ferntech','Fujifilm NZ','Kens Cameras & Video Ltd','Lacklands','Lazer Photos','Panasonic New Zealand Limited','Photo & Video International','Progear','Protel International Technologies Limited','Rubber Monkey','Snapshot Cameras','Wellington Photographic Supplies']
df_SO_unpaid_WS = df_SO_unpaid[df_SO_unpaid['Company'].isin(wholesaler)]
df_SO_unpaid_other = df_SO_unpaid[~df_SO_unpaid['Company'].isin(wholesaler)]

checker = df_SO_unpaid_other['Stage'] !="New"
df_SO_unpaid_other = df_SO_unpaid_other[checker]


df_SO_unpaid_WS.to_csv('SO follow up - Wholesaler - '+date.today().strftime("%d-%m-%Y")+'.csv')
df_SO_unpaid_other.to_csv('SO follow up - Other - '+date.today().strftime("%d-%m-%Y")+'.csv')

C:\Users\Roshan\AppData\Local\Temp\ipykernel_23828\3163378399.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SO_not_imported.loc[:,'Paid'] = df_SO_not_imported.loc[:,'Paid'].astype(str).apply(lambda x:x.replace('<font color=#009788><b>PAID: ','').replace('%</b></font>','').replace(' ','').replace('-','').replace('(','').replace(')','').replace('/','').replace('\t','').lower())
C:\Users\Roshan\AppData\Local\Temp\ipykernel_23828\3163378399.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SO_no

In [35]:
from email.policy import default
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from datetime import timedelta
from datetime import datetime

file_products = 'ProductExport-'+date.today().strftime("%d-%m-%Y")+'.csv'
file_PO = 'PO-'+date.today().strftime("%d-%m-%Y")+'.csv'

#set up product df
prod_col = ['Style Code', 'Brand', 'Code']
df_p = pd.read_csv(file_products, usecols=prod_col)

#set up PO df

po_col = ['Item Code', 'Item Name', 'Item Qty','Fully Received']
df_po = pd.read_csv(file_PO, usecols=po_col)

#get PO in the last 7 days

start_date = datetime.now()+ timedelta(days=-8)

df_po['Fully Received'] = pd.to_datetime(df_po['Fully Received'])
df_po = df_po.loc[df_po['Fully Received']>start_date]
df_po = df_po.loc[df_po['Item Qty']>0]

#merge with product info

df_final = df_po.merge(df_p, how='left', left_on='Item Code', right_on='Code')
df_final = df_final[['Style Code', 'Brand', 'Code','Item Name', 'Item Qty','Fully Received']]
df_final['Style Code'] = df_final['Style Code'].astype('float')
df_final = df_final.sort_values(by='Style Code',ascending=False)

df_final.to_csv('Products received in the last 7 days - '+date.today().strftime("%d-%m-%Y")+'.csv')

C:\Users\Roshan\AppData\Local\Temp\ipykernel_23828\4029504887.py:14: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_p = pd.read_csv(file_products, usecols=prod_col)
